In [49]:
# From https://gist.github.com/danoneata/49a807f47656fedbb389

import numpy as np


def fvecs_read(filename, c_contiguous=True):
    fv = np.fromfile(filename, dtype=np.float32)
    if fv.size == 0:
        return np.zeros((0, 0))
    dim = fv.view(np.int32)[0]
    assert dim > 0
    fv = fv.reshape(-1, 1 + dim)
    if not all(fv.view(np.int32)[:, 0] == dim):
        raise IOError("Non-uniform vector sizes in " + filename)
    fv = fv[:, 1:]
    if c_contiguous:
        fv = fv.copy()
    return fv

X = fvecs_read("../../data/sift/sift_base.fvecs")
data = X.T
data = data.astype(np.float64)

In [71]:
def choose_signal_and_atoms(data):
    N = len(data)
    signal_idx = np.random.choice(N)
    signals = data[signal_idx].reshape(1, -1)
    
    if signal_idx == 0:
        atoms = data[signal_idx+1:]
    elif signal_idx == N-1:
        atoms = data[:signal_idx]
    else:
        atoms = np.vstack([data[:signal_idx], data[signal_idx+1:]])  # TODO: Boundary

    return signals, atoms

In [75]:
from algorithms.action_elimination import action_elimination

for i in range(10):
    signals, atoms = choose_signal_and_atoms()
    best_atom, budget = action_elimination(atoms, signals, var_proxy = 5000, epsilon=0, delta=0.001, abs=True, batch_size=300)
    best_atom_naive = np.argmax(atoms @ signals.T)
    print(best_atom, best_atom_naive)

[[40]] 40
[[80]] 80
[[47]] 47
[[71]] 71
[[48]] 48
[[71]] 71
[[48]] 48
[[39]] 39
[[79]] 79
[[79]] 79
